In [11]:
import pandas as pd
import requests as rq

In [36]:
# Sample Request
municipality = 'Kappel am Albis'
api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
response = rq.get(api_url + municipality, headers={'X-Api-Key': '8AmHgsXCnbvi7hlJ0Mh/hQ==rt399afLziw2LIfI'})
if response.status_code == rq.codes.ok:
    print(response)
else:
    print("Error:", response.status_code, response.text)


<bound method Response.json of <Response [200]>>


In [40]:
response.json()[0]

{'name': 'Kappel am Albis',
 'latitude': 47.2288482,
 'longitude': 8.5251335,
 'country': 'CH',
 'state': 'Zurich'}

In [66]:
df_municipalities_done = pd.read_csv('Data/municipality_coordinates_in_progress.csv')
df_municipalities_done2 = pd.read_csv('Data/next_batch_municipality_coordinates_in_progress.csv')
done_mun1 = df_municipalities_done["Municipality"].tolist()
done_mun2 = df_municipalities_done2["Municipality"].tolist()
done_mun = done_mun1 + done_mun2
df_all_mun = pd.read_excel('Data/be-b-00.04-agv-20050313.xlsx', sheet_name='Gemeindeliste-Liste d. communes')
all_mun = df_all_mun["GDENAME"].tolist()
mun_todo = list(set(all_mun) - set(done_mun))
len(mun_todo)

1228

In [67]:
new_mun_coordinates = []
problems = []
len_todo = len(mun_todo)
for i, municipality in enumerate(mun_todo):
    api_url = f'https://api.api-ninjas.com/v1/geocoding?city={municipality}&country=Switzerland'
    response = rq.get(api_url + municipality, headers={'X-Api-Key': '8AmHgsXCnbvi7hlJ0Mh/hQ==rt399afLziw2LIfI'})
    if response.status_code == rq.codes.ok:
        if len(response.json())== 0:
            problems.append(municipality)
            print("Error with ", municipality)
            continue
        name = response.json()[0].get("name")
        latitude = response.json()[0].get("latitude")
        longitude = response.json()[0].get("longitude")
        new_mun_coordinates.append([name, latitude, longitude])
    else:
        print("Error:", response.status_code, response.text)
        problems.append(municipality)
    if i %50== 0:
        print(i, " of ", len_todo, " done")

columns = ['Municipality', 'Latitude', 'Longitude']
df_new_mun_coordinates = pd.DataFrame.from_records(new_mun_coordinates, columns=columns)
df_new_mun_coordinates.to_csv("Data/batch3_municipality_coordinates_in_progress")

0  of  1228  done
Error with  Oberhofen (AG)
100  of  1228  done
Error with  Willisau Land
Error with  Rüti (GL)
Error with  Riom-Parsonz
150  of  1228  done
Error with  Reckingen-Gluringen
200  of  1228  done
Error with  Villaz-Saint-Pierre
Error with  Gettnau
250  of  1228  done
Error with  Niedererlinsbach
300  of  1228  done
Error with  Niederried bei Kallnach
Error with  Obererlinsbach
Error with  Isorno
350  of  1228  done
400  of  1228  done
Error with  Casti-Wergenstein
450  of  1228  done
Error with  Tinizong-Rona
Error with  Unterbözberg
500  of  1228  done
Error with  Willisau Stadt
Error with  Wiesen (GR)
Error with  Schwanden (GL)
Error with  Münster-Geschinen
550  of  1228  done
Error with  St. Antönien Ascharina
600  of  1228  done
Error with  Heinrichswil-Winistorf
650  of  1228  done
700  of  1228  done
Error with  Tüscherz-Alfermée
750  of  1228  done
800  of  1228  done
850  of  1228  done
Error with  Klosters-Serneus
Error with  Thielle-Wavre
900  of  1228  done
Err

In [68]:
# combine all files
df_batch_1 = pd.read_csv('Data/municipality_coordinates_in_progress.csv')
df_batch_2 = pd.read_csv('Data/next_batch_municipality_coordinates_in_progress.csv')
df_batch_3 = pd.read_csv('../Data/batch3_municipality_coordinates_in_progress.csv')

In [69]:
df_combined = pd.concat([df_batch_1, df_batch_2, df_batch_3], ignore_index=True)

In [89]:
df_combined  = df_combined[['Municipality', 'Latitude', 'Longitude']]


In [83]:
len(all_mun)

2758

In [91]:
municipality_coordinates = pd.read_csv('../Data/municipality_to_latitude_longitude.csv')

In [92]:
municipality_coordinates.drop_duplicates(inplace=True)

In [93]:
municipality_coordinates[municipality_coordinates.duplicated()]

,Municipality,Latitude,Longitude


In [95]:
municipality_coordinates.to_csv('Data/municipality_to_latitude_longitude_noDuplicates.csv')

In [98]:
municipality_coordinates_no_dups = municipality_coordinates.drop_duplicates()
len(municipality_coordinates_no_dups)

2728